In [1]:
import os
import numpy as np
import pandas as pd
%matplotlib inline

breeding_sites = [ 
                'pot, flowerpot', 
                'stupa, tope', 
                'water jug', 
                'water bottle', 
                'ashcan, trash can, garbage can, wastebin, ash bin, ash-bin, ashbin, dustbin, trash barrel, trash bin', 
                'greenhouse, nursery, glasshouse', 
                'milk can', 
                'barrel, cask', 
                'canoe', 
                'rain barrel', 
                'lakeside, lakeshore', 
                'Dutch oven' 
                ]

all_train = pd.read_csv('train_test.csv', names=['img_name','1st_result','1st_score','2nd_result','2nd_score',
                                           '3rd_result','3rd_score','4th_result','4th_score',
                                           '5th_result','5th_score','segment','cls'])
all_train = all_train.drop('img_name', axis=1)
all_train.head()

,1st_result,1st_score,2nd_result,2nd_score,3rd_result,3rd_score,4th_result,4th_score,5th_result,5th_score,segment,cls
0,"dam, dike, dyke",0.079421,"solar dish, solar collector, solar furnace",0.057878,"lakeside, lakeshore",0.050353,"patio, terrace",0.047485,"seashore, coast, seacoast, sea-coast",0.030495,Car,0
1,"seashore, coast, seacoast, sea-coast",0.110984,"lakeside, lakeshore",0.067503,monastery,0.040836,"bell cote, bell cot",0.024820,street sign,0.024058,Road,0
2,"worm fence, snake fence, snake-rail fence, Vir...",0.111187,"mobile home, manufactured home",0.052045,pole,0.026348,"greenhouse, nursery, glasshouse",0.026229,"lumbermill, sawmill",0.023403,Pavement,0
3,"doormat, welcome mat",0.048527,broom,0.026041,sundial,0.019719,"patio, terrace",0.014585,parking meter,0.014006,Road,0
4,"maze, labyrinth",0.359694,"seashore, coast, seacoast, sea-coast",0.109345,"sandbar, sand bar",0.052152,"lakeside, lakeshore",0.025826,"leatherback turtle, leatherback, leathery turt...",0.024705,Road,0


In [25]:
def evaluate_any(all_train, breeding_site):
    use_any = all_train.loc[(all_train['1st_result'] == breeding_site) | 
                            (all_train['2nd_result'] == breeding_site) | 
                            (all_train['3rd_result'] == breeding_site) |
                            (all_train['4th_result'] == breeding_site) |
                            (all_train['5th_result'] == breeding_site)]
    return use_any

def evaluate_top1(all_train, breeding_site):
    use_top1 = all_train.loc[(all_train['1st_result'] == breeding_site)]
    return use_top1

def thresh(all_train, breeding_site):
    thresh = all_train[all_train['1st_score'] >= 0.15]
    thresh = thresh.loc[(all_train['1st_result'] == breeding_site)]
    return thresh

In [62]:
breeding_sites_acc = []
for breeding_site in  breeding_sites:
    evl_any = evaluate_any(all_train,  breeding_site)
    acc_any = float(format(np.mean(evl_any['cls']),'.7f'))
    
    evl_top1 = evaluate_top1(all_train,  breeding_site)
    acc_top1 = float(format(np.mean(evl_top1['cls']),'.7f'))
    
    evl_thresh = thresh(all_train,  breeding_site)
    acc_thresh = float(format(np.mean(evl_thresh['cls']),'.7f'))
       
    breeding_sites_acc.append([len(evl_any), acc_any, 
                               len(evl_top1), acc_top1,
                               len(evl_thresh), acc_thresh])
    
breeding_sites_acc = (np.transpose(breeding_sites_acc))
breeding_sites_acc = np.append(breeding_sites_acc,
                               [['Top-5_size'],['Top-5_acc'],
                                ['Top-1_size'],['Top-1_acc'],
                                ['Thresh_size'],['Thresh_acc'],
                               ], axis=1)

evaluation = pd.DataFrame.from_records(breeding_sites_acc, index='named', columns=breeding_sites+['named'])

In [63]:
all_train_top1 = all_train.loc[all_train['1st_result'].isin(breeding_sites)]
acc_all_train_top1 = format(np.mean(all_train_top1['cls']), '.7f')

all_train_top1_thresh_015 = all_train_top1[all_train_top1['1st_score'] >= 0.15]
acc_all_train_top1_thresh_015 = format(np.mean(all_train_top1_thresh_015['cls']), '.7f')

all_train_top1_thresh_030 = all_train_top1[all_train_top1['1st_score'] >= 0.30]
acc_all_train_top1_thresh_030 = format(np.mean(all_train_top1_thresh_030['cls']), '.7f')

acc_all_train_top5 = format(np.mean(all_train['cls']), '.7f')

all_train_evl = np.array([[len(all_train_top1_thresh_030),len(all_train_top1_thresh_015), len(all_train_top1),len(all_train)], 
                          [acc_all_train_top1_thresh_030, acc_all_train_top1_thresh_015, acc_all_train_top1, acc_all_train_top5]])

all_train_evl = np.append(all_train_evl,[['sample size'],['accuracy']], axis=1)

evaluation_all = pd.DataFrame.from_records(all_train_evl, index='named', 
                                           columns=['Top-1 & thresh=0.30','Top-1 & thresh=0.15','Top-1', 'Top-5', 'named'])


In [67]:
evaluation_all

,Top-1 & thresh=0.30,Top-1 & thresh=0.15,Top-1,Top-5
named,,,,
sample size,3739,7456,13274,60920
accuracy,0.6060444,0.5053648,0.3836824,0.2774458


In [65]:
evaluation

,"pot, flowerpot","stupa, tope",water jug,water bottle,"ashcan, trash can, garbage can, wastebin, ash bin, ash-bin, ashbin, dustbin, trash barrel, trash bin","greenhouse, nursery, glasshouse",milk can,"barrel, cask",canoe,rain barrel,"lakeside, lakeshore",Dutch oven
named,,,,,,,,,,,,
Top-5_size,11589.0,232.0,648.0,208.0,5587.0,9850.0,3017.0,1071.0,1737.0,2008.0,25291.0,496.0
Top-5_acc,0.4219519,0.2758621,0.4012346,0.4086538,0.2980132,0.3636548,0.4474644,0.5770308,0.3172136,0.5363546,0.2354593,0.4798387
Top-1_size,4236.0,33.0,48.0,40.0,925.0,1552.0,470.0,109.0,182.0,319.0,5292.0,68.0
Top-1_acc,0.4497167,0.4848485,0.4583333,0.475,0.4497297,0.3936856,0.5765957,0.559633,0.5879121,0.6677116,0.2673847,0.5441176
Thresh_size,2862.0,17.0,21.0,19.0,476.0,961.0,229.0,67.0,95.0,206.0,2471.0,32.0
Thresh_acc,0.5345912,0.7647059,0.7142857,0.5263158,0.6491597,0.4412071,0.8165939,0.7014925,0.6526316,0.7621359,0.402671,0.59375
